In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm

%matplotlib inline
from IPython.core.pylabtools import figsize

In [1]:
from snorkel.labeling.preprocess import preprocessor
from snorkel.labeling.apply import PandasLFApplier
from snorkel.labeling.lf import labeling_function
from snorkel.types import DataPoint

POS = 1
NEG = -1
ABSTAIN = 0

In [4]:
#Load in our cleaned data and re-check category labels
media_df = pd.read_csv('/Users/awhite/Documents/snorkel/arabic_news_cleaned.csv')
media_df.category.value_counts()

كرة_القدم                       4301
الأزمة_السورية                 3160
جماعات_مسلحة                    2142
أسواق_النفط                    1364
لاجئون                         1256
رياضات_اخرى                     1100
المعارضة_السورية                1002
صواريخ                           741
التقنية_والمعلومات               639
فضاء                             630
أسلحة_ومعدات_عسكرية             625
الهجرة_إلى_أوروبا              616
الأزمة_الأوكرانية              595
اكتشافات                         577
الأزمة_اليمنية                  566
تفجيرات                          559
مشاهير                           539
البحوث_الطبية                    530
معلومات_عامة                     484
طائرات_حربية                    479
الانتخابات_الأمريكية            447
جرائم                           433
مظاهرات                          422
مؤشرات_اقتصادية                 400
عقوبات_اقتصادية                  397
الاعتراف_بدولة_فلسطين            369
امراض                            323
أولمبياد_ريو_د

There are several broad categories like جماعات_مسلحة (armed groups) and أسواق_النفط (oil markets), then more specific categories such as two for the Syrian conflict: الأزمة_السورية (Syrian conflict) and المعارضة_السورية (Syrian opposition). Let's start with one of the general categories, then more to Syria.  

In [ ]:
@labeling_function
def oil_terms(x):
    if "نفظ|برتول" in x.text:
        return POS
    else:
        return ABS

In [ ]:
@labeling_function